In [1]:
using PorousMaterials # Pkg.clone("https://github.com/SimonEnsemble/PorousMaterials.jl", "v0.1.1")
using DelimitedFiles
using LinearAlgebra
using JLD2
using CoherentPointDrift
import Bio3DView
using Printf
import PyPlot
using Random
Random.seed!(1337); # set random number seed for complete reproducibility

# Database of cages

Cage database CDB41 (41 cages) from [here](https://github.com/marcinmiklitz/CDB41). Cleaned of solvent. Thanks to Kim Jelfs and M. Miklitz for kindly providing the cages.

Reference:
> M. Miklitz, S. Jiang, R. Clowes, M. E. Briggs, A. I. Cooper and K. E. Jelfs, Computational Screening of Porous Organic Molecules for Xenon/Krypton Separation, J. Phys. Chem. C, 2017. DOI: 10.1021/acs.jpcc.7b03848

Other set of 33 cages from the .xyz files deposited as ESI of DOI 10.1038/s41467-018-05271-9

> R. L. Greenaway, V. Santolini, M. J. Bennison, B. M. Alston, C. J. Pugh, M. A. Little, M. Miklitz, E. G. B. Eden-Rump, R. Clowes, A. Shakil, H. J. Cuthbertson, H. Armstrong, M. E. Briggs, K. E. Jelfs & A. I. Cooper. High-throughput discovery of organic cages and catenanes using computational screening fused with robotic synthesis. Nature Communications, 2018. DOI: 10.1038/s41467-018-05271-9
                                                              
I manually looked at Fig 4 in the main text and selected those that were synthesized. Some cages required visualization to compare to Fig 4 since e.g. there are multiple versions of B23, a few of which were apparently not synthesized but only computational predictions.

The `.xyz` files describing the molecular structure of all of these cages are in the directory `all_cages`.

In [2]:
cages = readdlm("all_cages/all_cages.txt", String)[:];

# Center cages

The center of mass will be set to be the origin.
Each cage will be rotated such that it is aligned with its principle moments of inertia.
i.e. the moment of inertia matrix of an "aligned" cage is diagonal.


First, a function to compute the center of mass of a group of `atoms` at Cartesian coordinates `x`, stored in the columns.

In [3]:
function center_of_mass(atoms::Array{Symbol, 1}, x::Array{Float64, 2})
    ams = read_atomic_masses()
    com = [0.0, 0.0, 0.0]
    total_mass = 0.0
    for (i, atom) in enumerate(atoms)
        com += ams[atom] * x[:, i]
        total_mass += ams[atom]
    end
    return com / total_mass
end

center_of_mass (generic function with 1 method)

In [4]:
function centered_cage_coords(cage::AbstractString)
    # read in raw .xyz from `all_cages`
    atoms, x = read_xyz("all_cages/" * cage * ".xyz")
    
    # compute center of mass
    x_com = center_of_mass(atoms, x)
    
    # shift coords so that cage is centered
    x = x .- x_com
    
    # write centered cage coords
    write_xyz(atoms, x, "centered_cages/" * cage * ".xyz")
    
    return atoms, x
end

centered_cage_coords (generic function with 1 method)

# Porosity point clouds

In [5]:
function generate_porosity_point_cloud(atoms::Array{Symbol, 1}, 
                                       x::Array{Float64, 2}, 
                                       nb_pts_in_porosity_cloud::Int, 
                                       snapshot_radius::Union{Nothing, Float64};
                                       gaussian_scaler::Float64=3.0)
    # store points in void space here.
    x_porosity_cloud = zeros(3, nb_pts_in_porosity_cloud)

    # determine if void space via potential energy of He
    he = Molecule("He")
    ljff = LJForceField("UFF.csv", cutoffradius=14.0, mixing_rules="geometric")
    ljspheres = Atoms(atoms, x)

    # compute radius of the molecule; this determines variance of Gaussian if snapshot size not provided
    if snapshot_radius == nothing
        snapshot_radius = maximum([norm(x[:, a]) for a = 1:length(atoms)])
    end
    
    n_hits = 0
    while n_hits < nb_pts_in_porosity_cloud
        # bias insertions towards center
        x_insert = randn(3) * snapshot_radius / gaussian_scaler # scale smaller to bias describing core more.
        if norm(x_insert) > snapshot_radius
            continue
        end

        # put helium at this grid pt
        translate_to!(he, x_insert)

        # compute potential energy of He adsorbate here
        energy = vdw_energy_no_PBC(he, ljspheres, ljff)

        if energy < 298.0
            n_hits += 1
            x_porosity_cloud[:, n_hits] = x_insert
        end
    end
    
    return x_porosity_cloud
end

# atoms, x = centered_cage_coords("B1")
# x = generate_porosity_point_cloud(atoms, x, 10000, nothing)
# write_xyz([:H for i = 1:10000], x, "test.xyz")

generate_porosity_point_cloud (generic function with 1 method)

# Rotate to align principal axes of rotation with Cartesian axes

Next, a function to compute the moment of inertia matrix of a group of `atoms` at Cartesian positions `x`. 

See <a href="https://chem.libretexts.org/Textbook_Maps/Physical_and_Theoretical_Chemistry_Textbook_Maps/Map%3A_Physical_Chemistry_(McQuarrie_and_Simon)/13%3A_Molecular_Spectroscopy/13-08._The_Rotational_Spectrum_of_a_Polyatomic_Molecule_Depends_Upon_the_Principal_Moments_of_Inertia_of_the_Molecule">Chemistry Libre Texts</a> moment of inertia formulas.

In [6]:
function moment_of_inertia(x::Array{Float64, 2})
    # moment of inertia matrix `mi`
    mi = zeros(Float64, 3, 3)
    for i = 1:3
        for j = 1:3
            for a = 1:size(x)[2]
                if i == j
                    ids = collect(setdiff(Set([1, 2, 3]), Set([i])))
                    mi[i, j] += sum(x[ids, a] .^ 2)
                else
                    mi[i, j] -= x[i, a] * x[j, a]
                end
            end
        end
    end
    
    # should be symmetric!
    @assert isapprox(mi, mi')
        
    return mi
end

moment_of_inertia (generic function with 1 method)

In [7]:
viewcage(cage::AbstractString) = Bio3DView.viewfile("final_aligned_cages/" * cage * ".xyz", "xyz")

viewcage (generic function with 1 method)

In [8]:
write_final_aligned_cage(atoms::Array{Symbol, 1}, x::Array{Float64, 2}, cage::AbstractString) = write_xyz(atoms, x, "final_aligned_cages/" * cage * ".xyz")
read_final_aligned_cage(cage::AbstractString) = read_xyz(atoms, x, "final_aligned_cages/" * cage * ".xyz")

write_rotational_inertia_aligned_cage(atoms::Array{Symbol, 1}, x::Array{Float64, 2}, cage::AbstractString) = write_xyz(atoms, x, "rotational_inertia_aligned_cages/" * cage * ".xyz")
read_rotational_inertia_final_aligned_cage(cage::AbstractString) = read_xyz("rotational_inertia_aligned_cages/" * cage * ".xyz")

read_rotational_inertia_final_aligned_cage (generic function with 1 method)

In [ ]:
nb_pts_in_pt_cloud = 100000

aligned = Dict(cage => false for cage in cages)

for cage in cages
    # read in centered cage
    atoms, x = centered_cage_coords(cage)
    
    # generate porosity point cloud
    x_pt_cloud = generate_porosity_point_cloud(atoms, x, nb_pts_in_pt_cloud, nothing) # use radius of molecule for this.
    write_xyz([:H for i = 1:nb_pts_in_pt_cloud], x_pt_cloud, 
        "centered_cages/porosity_point_clouds/" * cage * ".xyz")
    
    # diagnomize moment of inertia matrix
    mi = moment_of_inertia(x_pt_cloud)
    λ, v = eigen(mi) # columns of v are eigenvalues
    ids = sortperm(λ, rev=true) # sort eigenvalues from large to small.
    λ = λ[ids]
    v = v[:, ids]
    # tests out of paranoia
    @assert(isapprox(mi * v, v * diagm(0 => λ)), "eigenvectors not right")
    @assert(isapprox([norm(v[:, i]) for i = 1:3], ones(3)), "eigenvectors not unit vectors")
    @assert(isapprox(mi, v * diagm(0 => λ) * v'))
    # `v` is a rotation matrix b/c `mi` is symmetric and the columns are orthonormal
    # i.e. `v` is a unitary matrix. now use it to rotate x to align with principle axes of rotation
    @assert(isapprox(v' * v, Diagonal{Float64}(I, 3)), "v is not unitary...")
    @assert (λ[1] >= λ[2]) & (λ[2] >= λ[3])
    
    # align principal axes of rotation of the cage with the Cartesian axes
    write_rotational_inertia_aligned_cage(atoms, v' * x, cage)
    
    # look for degeneracy
    xy_symmetry = isapprox(λ[1], λ[2], rtol=0.01) # can use rtol b/c fixed # pts now.
    yz_symmetry = isapprox(λ[2], λ[3], rtol=0.01)
    if xy_symmetry || yz_symmetry
        printstyled(cage * " has nearly degenerate axes of inertia\n", color=:red)
    else
        aligned[cage] = true
        write_final_aligned_cage(atoms, v' * x, cage)
        printstyled(cage * " has unique axes of inertia\n", color=:green)
    end
end

A11 has nearly degenerate axes of inertia
B11 has nearly degenerate axes of inertia


Write to file list of cages where principal axes of inertia are authoratative on how to align.

In [ ]:
rotational_inertia_aligned_cages = cages[[aligned[cage] for cage in cages]]
@save "rotational_inertia_aligned_cages.jld2" rotational_inertia_aligned_cages

# Rigid point set registration to align cages consistently where moments of inertia about principal axes are nearly degenerate

Write porosity point clouds of rotational-inertia-aligned cages for further alignment where principal axes are nearly dgenerate. Store in `rotational_inertia_aligned_cages/porosity_point_clouds`.

In [ ]:
read_porosity_pt_cloud_rot_aligned(cage::AbstractString) = read_xyz("rotational_inertia_aligned_cages/porosity_point_clouds/" * cage * ".xyz")[2]
write_porosity_pt_cloud_rot_aligned(x_pt_cloud::Array{Float64, 2}, cage::AbstractString) = write_xyz(
    [:H for i = 1:nb_pts_in_pt_cloud], 
    x_pt_cloud, 
    "rotational_inertia_aligned_cages/porosity_point_clouds/" * cage * ".xyz")

In [ ]:
nb_pts_in_pt_cloud = 5000

porosity_point_clouds = Dict{AbstractString, Array{Float64, 2}}()

# use same dist'n of points for this one to compare cages easily
for cage in cages
    atoms, x = read_rotational_inertia_final_aligned_cage(cage)
    
    porosity_point_clouds[cage] = generate_porosity_point_cloud(atoms, x, nb_pts_in_pt_cloud, nothing, gaussian_scaler=5.0)
    
    write_porosity_pt_cloud_rot_aligned(porosity_point_clouds[cage], cage)
end

In [ ]:
function find_pair_to_align(cage_ys::Array{String, 1}, cage_xs::Array{String, 1})
    # look at all possible pairs
    pairz = Tuple{String, String}[]
    ℓs = Float64[]
    for cage_y in cage_ys
        for cage_x in cage_xs
            if cage_y == cage_x
                continue
            end
            
            jldfilename = jldopen(@sprintf("cpd_results/align_%s_to_%s.jld2", cage_y, cage_x))
            ℓ = read(jldfilename, "ℓ")
            close(jldfilename)
            
            push!(ℓs, ℓ)
            push!(pairz, (cage_y, cage_x))
        end
    end
    best_pair = pairz[argmin(ℓs)]
    return best_pair[1], best_pair[2]
end

In [ ]:
aligned = Dict(cage => false for cage in cages)
nb_pts_in_pt_cloud = 500

cage_y, cage_x = find_pair_to_align(cages, cages)
aligned[cage_x] = true

while sum([! aligned[cage] for cage in cages]) > 0
    
    aligned_cages = cages[[aligned[cage] for cage in cages]]
    unaligned_cages = cages[[! aligned[cage] for cage in cages]]
    
    cage_y, cage_x = find_pair_to_align(unaligned_cages, aligned_cages)
    printstyled(@sprintf("Aligning cage %s to %s...\n", cage_y, cage_x), color=:yellow)
    
    atoms_x, x = read_rotational_inertia_final_aligned_cage(cage_x)
    x_pt_cloud = generate_porosity_point_cloud(atoms_x, x, nb_pts_in_pt_cloud, nothing)
    
    atoms_y, y = read_rotational_inertia_final_aligned_cage(cage_y)
    y_pt_cloud = generate_porosity_point_cloud(atoms_y, y, nb_pts_in_pt_cloud, nothing)
    
    # transformation is applied not to reference cage but the unaligned cage!
    R, t, σ², ℓ = CoherentPointDrift.rigid_point_set_registration(x_pt_cloud, y_pt_cloud, verbose=false,
        w=0.0, σ²_tol=0.1, q_tol=1.0, max_nb_em_steps=25, print_ending=true)
    
    write_final_aligned_cage(atoms_y, R * y, cage_y)
    
    aligned[cage_y] = true
end

In [ ]:
for cage in cages
    # if cage aligned uniquely with axes of inertia, carry on to the next cage.
    if aligned[cage]
        printstyled(cage * " aligned via principal axes of inertia\n", color=:yellow)
        continue
    end
    
    # cage is unaligned. find best to align to among already-aligned cages
    printstyled("Aligning " * cage * " to ...", color=:green)

    aligned_cages = cages[[aligned[cage] for cage in cages]]

    # find best cage to align to
    align_to = find_best_cage_to_align_to(cage, aligned_cages, read_from_file=true)# , cage_names=aligned_cages)
    printstyled(align_to * "\n", color=:red)
    println("\tCoherent point drift params: ")
    
    x = read_porosity_pt_cloud_rot_aligned(align_to)
    y = read_porosity_pt_cloud_rot_aligned(cage)
    
    R, t, σ², ℓ = CoherentPointDrift.rigid_point_set_registration(x, y, verbose=false,
            w=0.0, σ²_tol=0.1, q_tol=1.0, max_nb_em_steps=25, print_ending=true)
    
    atoms, y = centered_cage_coords(cage)
    write_final_aligned_cage(atoms, R * y, cage)
    aligned[cage] = true
end

In [ ]:
# cage_x = "B2"
# cage_y = "C2"

# atoms_x, x = read_rotational_inertia_final_aligned_cage(cage_x)
# x_pt_cld = generate_porosity_point_cloud(atoms_x, x, 5000, nothing)
# atoms_y, y = read_rotational_inertia_final_aligned_cage(cage_y)
# y_pt_cld = generate_porosity_point_cloud(atoms_y, y, 5000, nothing)

# R, t, σ², ℓ = CoherentPointDrift.rigid_point_set_registration(x_pt_cld ,y_pt_cld, verbose=true,
#             w=0.0, σ²_tol=0.1, q_tol=1.0, max_nb_em_steps=25, print_ending=true)

# write_final_aligned_cage(atoms_y, R * y, cage_y)